El uso de Python como apoyo al pesaje de vehículos de carga en movimiento (WIM)
============================================


Muchos accidentes en carreteras son causados, directamente o indirectamente, por vehículos de carga conducidos con sobrepeso. Estos damnifican más el pavimento y también sufren más efectos dinámicos durante las curvas.

Para inhibir el exceso de peso de los vehículos de carga, es necesario fiscalizar estas infracciones y, cuando necesario, aplicar las medidas establecidas por ley, como multas y aprehensiones. Un método que está siendo investigado en muchas partes del mundo es el pesaje en movimiento. Este método tiene como ventajas la economía en espacio físico y operación, ya que sus sensores son implantados en la propia carretera, y no implica en atrasos directos en el flujo de la vía, pues puede pesar los camiones transitando en alta velocidad.

En el proyecto desarrollado en el laboratorio de transportes (LabTrans) de la Universidade Federal de Santa Catarina (UFSC), el lenguaje Python fue adoptado para substituir el sistema anterior hecho en LabView. El motivo principal de esta migración fue la utilización de un lenguaje con estructuras transparentes, distinto del LabView (este trabaja con programación en bloques y, a veces, de acceso restricto.

El lenguaje Python es potenciado por las innumerables librerías de apoyo que le proporciona un mejor desempeño (a veces muy cerca de lenguajes como C y Java) y mayor comodidad para manipular los datos (como en el lenguaje R).

Las librerías utilizadas en este proyecto fueron: numpy, scipy, flask, pandas, pandasql, sqlalchemy, statsmodels, numba, pymodelfit, scikit-learn, pydaqmx.


Descripción del proyecto
----------------------------

Un sistema computacional de pesaje de vehículos en movimiento está compuesto, básicamente, de:
- Adquisición de señal (de los sensores de peso en la vía);
- Segmentación de señal (para recortar la señal respectiva al camión medido);
- Tratamiento de señales;
- Cálculos (velocidad, número de ejes, grupos de ejes, distancia entre ejes, peso total, peso por ejes, peso por grupo de ejes, largo);
- Clasificación del vehículo;
- Calibración;
- Reconocimiento de matrículas vehiculares;
- Detección de infracción;

Es necesario que el sistema sea rápido y robusto para procesar todas estas informaciones en el menor tiempo posible. Por el Python no ser un lenguaje reconocido por tener un alto desempeño, es necesario utilizar librerías y métodos para potenciar su capacidad de procesamiento.

La adquisición de datos fue hecha através de placas de adquisición DAQmx de la empresa National Instruments. Para comunicar con estas fue utilizada la librería PyDAQmx, un wrap hecho en Python para los controladores del hardware fornecidos por la empresa. Esta librería es una interfaz completa para los controladores NIDAQmx ANSI C e importa todas las funciones del controlador e importa todas las constantes predefinidas. Como resultado, la librería retorna un objeto "numpy.array".

Después de adquir la señal de los sensores, el sistema la almacena en un buffer circular en memoria que, dentro un proceso paralelo, es analisada en busca de una señal completa de un vehículo (segmento). Este proceso fue construído de manera muy simple, donde el programa espera la señal desde un bucle inductivo y, cuando accionado, segmenta la señal con valores respectivos a los 3 segundos siguientes. 

Después de segmentados, los datos brutos son almacenados en la base de datos, posibilitando cambiar los métodos de cálculos o parámetros de calibración, a cualquier momento de la ejecución del proyecto, a fin de obtener el mejor resultado.

Para la realización de los cálculos, la señal necesita ser tradada (filtro de señal) y, para eso, fue aplicado un filtro pasa baja de órden 1, a 600 Hz, utilizando los métodos filtfilt y butterworth de la librería scipy. En esta etapa, los datos ya están almacenados en un objeto "pandas.DataFrame".

En todos los métodos y funciones de cálculos en el sistema, el tipo patrón para los conjuntos de datos es el "pandas.DataFrame". Este es utilizado desde el momento de la lectura en la base de datos, en conjunto con sqlalchemy, hasta en los cálculos, ploteos y grabación en base de datos o en archivos CSV. el "pandas.DataFrame" fornece mecanismos para manipulación de datos muy parecidos a los utilizados en el lenguaje R.

El método definido para clasificar el tipo de veículo medido presentava muy bajo desempeño. Para solucionar este problema fue utilizada la librería numba, reduciendo el tiempo de procesamiento en cerca de unos 80%.

El reconocimiento de matrículas vehiculares fue realizado através de una cámara ALPR, de la empresa ARH. Con finalidad de medir la eficiencia de los resultados, fueron probadas algunas librerías libres para el tema. Uno de los materiales más interesantes encontrados a respecto fue el trabajo del investigador K.M. Sajjad (Automatic License Plate Recognition using Python and OpenCV) que fornece informaciones y ejemplos de cómo hacer el reconocimiento de matrículas vehículares utilizando el lenguaje Python.

La calibración períodicas en sistemas de pesaje es muy importante para mantener un bajo margen de errores de los pesos calculados. Para apoyar esta etapa fue utilizado el método de regresión lineal por mínimos cuadrados (OLS) de la librería "statsmodels" que, por ejemplo, posibilita saber informaciones como el coeficiente de determinación (R²) resultante de la regresión lineal. La librería scikit-learn también fue utilizada en esta etapa con finalidad de apoyo en la análisis de los resultados. Através de los datos de peso calculados para cada sensor (en el tramo reservado para los sensores piezoeléctrico cuarzo fueron instaladas 16 líneas de sensores), fueron eligidos algunos modelos matemáticos en scikit-learn para predecir el valor del peso bruto total (el sistema calcula através de media ponderada). Através de esta librería, es posible medir y evaluar los resultados de los modelos y, posteriormente, compararlos entre sí.

Con base en los resultados del pesaje, clasificación y reconocimiento de la matrícula vehicular es posible saber si el vehículo cometió alguna infracción y, en caso positivo, es posible vincular la infracción a la identificación del vehículo infractor.

